# Genetic Algorithm (Knapsack Problem)

This project implements a genetic algorithm to solve the knapsack problem. The objective is to select a subset of items to maximize their total utility while satisfying multiple cost constraints.

In [3]:
import numpy as np

In [12]:
class GeneticAlgorithmMDKP:
    def __init__(self, num_objects, utilities, costs, budgets, pop_size=100, generations=1000, mutation_rate=0.01, crossover_rate=0.7, init_method='random'):
        self.num_objects = num_objects
        self.utilities = utilities
        self.costs = costs
        self.budgets = budgets
        self.pop_size = pop_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.init_method = init_method
        self.population = self.initialize_population()


    def initialize_population(self):
        if self.init_method == 'random':
            return self.random_initialization()
        elif self.init_method == 'greedy':
            return self.greedy_initialization()
        elif self.init_method == 'mixed':
            return self.mixed_initialization()
        else:
            raise ValueError("Invalid initialization method.")

    def random_initialization(self):
        population = []
        while len(population) < self.pop_size:
            individual = np.random.randint(2, size=self.num_objects)
            if self.is_feasible(individual):
                population.append(individual)
        return np.array(population)

    def greedy_initialization(self):
        population = []
        utility_cost_ratio = self.utilities / np.sum(self.costs, axis=1)
        sorted_indices = np.argsort(utility_cost_ratio)[::-1]

        for _ in range(self.pop_size):
            individual = np.zeros(self.num_objects, dtype=int)
            total_costs = np.zeros(len(self.budgets))

            for idx in sorted_indices:
                if np.all(total_costs + self.costs[idx] <= self.budgets):
                    individual[idx] = 1
                    total_costs += self.costs[idx]

            if self.is_feasible(individual):
                population.append(individual)

        return np.array(population)

    def mixed_initialization(self):
        population = []
        half_pop_size = self.pop_size // 2

        # Random initialization
        while len(population) < half_pop_size:
            individual = np.random.randint(2, size=self.num_objects)
            if self.is_feasible(individual):
                population.append(individual)

        # Greedy initialization
        utility_cost_ratio = self.utilities / np.sum(self.costs, axis=1)
        sorted_indices = np.argsort(utility_cost_ratio)[::-1]

        while len(population) < self.pop_size:
            individual = np.zeros(self.num_objects, dtype=int)
            total_costs = np.zeros(len(self.budgets))

            for idx in sorted_indices:
                if np.all(total_costs + self.costs[idx] <= self.budgets):
                    individual[idx] = 1
                    total_costs += self.costs[idx]

            if self.is_feasible(individual):
                population.append(individual)

        return np.array(population)

    def is_feasible(self, individual):
        for i in range(len(self.budgets)):
            if np.sum(individual * self.costs[:, i]) > self.budgets[i]:
                return False
        return True

    def fitness(self, individual):
        return np.sum(individual * self.utilities)

    def selection(self):
        fitnesses = np.array([self.fitness(ind) for ind in self.population])
        probabilities = fitnesses / fitnesses.sum()
        selected_indices = np.random.choice(len(self.population), size=len(self.population), p=probabilities)
        return self.population[selected_indices]

    def crossover(self, parent1, parent2):
        if np.random.rand() < self.crossover_rate:
            point = np.random.randint(1, self.num_objects-1)
            child1 = np.concatenate((parent1[:point], parent2[point:]))
            child2 = np.concatenate((parent2[:point], parent1[point:]))
            return child1, child2
        else:
            return parent1, parent2

    def mutate(self, individual):
        for i in range(len(individual)):
            if np.random.rand() < self.mutation_rate:
                individual[i] = 1 - individual[i]
        return individual

    def repair(self, individual):
        total_costs = np.sum(individual[:, np.newaxis] * self.costs, axis=0)
        if np.all(total_costs <= self.budgets):
            return individual
        
        sorted_indices = np.argsort(self.utilities / np.sum(self.costs, axis=1))[::-1]
        for idx in sorted_indices:
            if individual[idx] == 1 and any(total_costs > self.budgets):
                individual[idx] = 0
                total_costs -= self.costs[idx]
        
        for idx in sorted_indices:
            if individual[idx] == 0 and all(total_costs + self.costs[idx] <= self.budgets):
                individual[idx] = 1
                total_costs += self.costs[idx]
        
        return individual

    def run(self):
        best_solution = None
        best_fitness = 0
        
        for generation in range(self.generations):
            new_population = []
            selected_population = self.selection()
            
            for i in range(0, len(self.population), 2):
                parent1, parent2 = selected_population[i], selected_population[i+1]
                child1, child2 = self.crossover(parent1, parent2)
                new_population.append(self.mutate(self.repair(child1)))
                new_population.append(self.mutate(self.repair(child2)))
            
            self.population = np.array(new_population)
            current_best = max(self.population, key=self.fitness)
            current_fitness = self.fitness(current_best)
            
            if current_fitness > best_fitness:
                best_solution, best_fitness = current_best, current_fitness
            
            # print(f"Generation {generation+1}: Best Fitness = {best_fitness}")
        
        return best_solution, best_fitness

## Tests

The following ceode cell runs the genetic algorithm ten times with different random initialisations and stores the results in a dataframe

There are 3 possible initialisation techniques:

1. **'random'** Initialize some individuals based on a greedy approach, where items with the highest utility-to-cost ratio are added until no more items can be added without violating the constraints.

2. **'greedy'**
Initialize some individuals based on a greedy approach, where items with the highest utility-to-cost ratio are added until no more items can be added without violating the constraints.

3. **'mixed'** 
Combine random and greedy initialization to create a more diverse initial population.

In [13]:
import pandas as pd

num_runs = 10

def run_genetic_algorithm():
    num_objects = 10
    utilities = np.random.randint(1, 100, size=num_objects)
    costs = np.random.randint(1, 50, size=(num_objects, 3))
    budgets = np.random.randint(50, 150, size=3)
    ga = GeneticAlgorithmMDKP(num_objects, utilities, costs, budgets, init_method='greedy')
    best_solution, best_fitness = ga.run()
    return best_solution, best_fitness

results = []

for i in range(num_runs):
    best_solution, best_fitness = run_genetic_algorithm()
    
    results.append({'Best Solution': best_solution,
                    'Best Fitness': best_fitness})

greedy_results_df = pd.DataFrame(results)

In [8]:
random_results_df.head(10)

,Best Solution,Best Fitness
0,"[0, 1, 1, 0, 1, 1, 1, 0, 1, 0]",391
1,"[0, 1, 1, 1, 0, 0, 1, 1, 1, 1]",368
2,"[0, 1, 1, 0, 1, 1, 1, 0, 0, 1]",374
3,"[1, 0, 1, 1, 0, 0, 0, 0, 1, 1]",388
4,"[1, 1, 0, 1, 0, 1, 0, 0, 1, 1]",256
5,"[0, 1, 0, 1, 0, 1, 1, 1, 0, 0]",343
6,"[1, 0, 1, 1, 1, 1, 1, 0, 0, 0]",379
7,"[1, 1, 1, 0, 1, 1, 0, 0, 0, 1]",434
8,"[1, 0, 0, 0, 0, 1, 1, 1, 1, 1]",486
9,"[1, 0, 1, 0, 0, 1, 1, 0, 1, 1]",338


In [11]:
greedy_results_df.head(10)

,Best Solution,Best Fitness
0,"[0, 1, 1, 0, 1, 0, 0, 1, 0, 1]",287
1,"[1, 1, 0, 0, 0, 0, 1, 1, 0, 1]",370
2,"[0, 0, 0, 0, 1, 1, 1, 1, 0, 1]",312
3,"[1, 0, 0, 1, 0, 1, 0, 1, 1, 1]",248
4,"[1, 0, 1, 0, 0, 0, 1, 0, 1, 1]",341
5,"[1, 0, 0, 1, 1, 1, 0, 1, 0, 1]",341
6,"[0, 1, 0, 0, 0, 1, 1, 1, 1, 1]",271
7,"[0, 1, 0, 1, 1, 1, 1, 0, 0, 1]",358
8,"[1, 0, 1, 0, 0, 1, 1, 0, 1, 0]",313
9,"[0, 0, 0, 1, 0, 1, 1, 0, 1, 1]",355


In [6]:
mixed_results_df.head(10)

,Best Solution,Best Fitness
0,"[1, 1, 1, 1, 0, 1, 0, 0, 0, 0]",304
1,"[1, 1, 1, 0, 1, 1, 1, 0, 0, 1]",488
2,"[0, 0, 1, 0, 0, 1, 0, 1, 1, 1]",343
3,"[1, 1, 1, 0, 1, 1, 1, 0, 1, 1]",496
4,"[1, 1, 1, 1, 1, 0, 1, 0, 0, 0]",405
5,"[1, 1, 1, 1, 1, 0, 1, 0, 0, 1]",521
6,"[0, 0, 1, 0, 1, 1, 1, 1, 0, 1]",461
7,"[1, 0, 0, 0, 1, 1, 1, 1, 0, 1]",243
8,"[0, 1, 1, 1, 0, 0, 1, 1, 1, 0]",481
9,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1]",624


## Citation

```
@article{chu1998genetic,
  title={A genetic algorithm for the multidimensional knapsack problem},
  author={Chu, Paul C and Beasley, John E},
  journal={Journal of heuristics},
  volume={4},
  pages={63--86},
  year={1998},
  publisher={Springer}
}
```